- LangChain 라이브러리를 설치하는 명령어입니다. LangChain은 LLM(대형 언어 모델)과 함께 작업할 때 유용한 도구들을 제공하는 파이썬 라이브러리입니다.

In [3]:
!pip install langchain

- OpenAI의 Python 라이브러리를 설치하는 명령어입니다. 이 라이브러리는 OpenAI의 API와 상호작용하기 위해 필요합니다.

In [2]:
!pip install openai

- os 모듈을 불러오고, OpenAI API 키를 환경 변수 OPENAI_API_KEY로 설정합니다. 이 키를 통해 OpenAI API에 접근할 수 있습니다.
- langchain_openai 모듈에서 ChatOpenAI 클래스를 불러옵니다.
- llm이라는 변수에 ChatOpenAI 객체를 할당합니다. 이 객체는 OpenAI의 GPT 모델을 사용하며, temperature=0은 출력의 창의성을 낮추기 위해 설정합니다. 모델 이름은 gpt-4o-mini로 설정되어 있습니다.

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hXedXDpBrlTcFzTzuwsuT3BlbkFJNgLSTtQgf19RNc0i7K5b" #openai 키 입력

#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
                 model_name='gpt-4o-mini',  # 모델명 #gpt-4o-mini #gpt-3.5-turbo-16k
                )
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000020714D0D160> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020714D15AC0> model_name='gpt-4o-mini' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''


- 대화 기록을 관리하고 사용하는 데 필요한 클래스를 불러옵니다.
- RunnableWithMessageHistory는 LLM을 사용한 대화 세션을 기록하고 관리하기 위해 사용되는 클래스입니다.
- BaseChatMessageHistory와 InMemoryChatMessageHistory는 대화 내역을 저장하는 데 사용됩니다.
- 대화 세션 기록을 저장하기 위한 store라는 딕셔너리를 초기화합니다. 이 딕셔너리는 세션 ID를 키로 하고, 그에 해당하는 대화 기록을 값으로 저장합니다.

In [5]:
# https://python.langchain.com/v0.2/docs/tutorials/chatbot/
#from langchain import ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

print(store)

{}


- session_id를 받아 해당 세션의 대화 기록을 반환하는 함수입니다.
- 만약 session_id에 해당하는 기록이 없으면 새로운 InMemoryChatMessageHistory 객체를 생성해 store에 저장합니다.

In [6]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

- 주석 처리된 ConversationChain 대신 RunnableWithMessageHistory를 사용하여 대화 체인을 구성합니다.
- conversation 객체는 RunnableWithMessageHistory로 초기화되며, 여기서 llm을 사용하고 대화 기록을 관리하기 위해 get_session_history 함수를 전달합니다.

In [7]:
#conversation = ConversationChain(llm=llm, verbose=True)
# RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history
)
print(conversation)

bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020714D0D160>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020714D15AC0>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), config_factories=[<function Runnable.with_alisteners.<locals>.<lambda> at 0x00000207125ABA60>]))], default=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020714D0D160>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020714D15AC0>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), 

- 주석 처리된 부분 이전에 사용되었던 predict 메서드를 대체하는 코드입니다.
- session_id라는 변수에 고유한 세션 ID를 할당합니다. 이 ID를 통해 대화 세션을 식별하고, 해당 세션의 대화 기록을 유지할 수 있습니다.
- invoke 메서드를 사용하여 대화 세션을 진행합니다.
- 각 대화 세션의 출력 결과를 출력합니다. output1, output2, output3 각각의 내용을 확인하여 모델이 입력에 대해 어떻게 응답했는지 확인할 수 있습니다.
- store 딕셔너리의 내용을 출력하여, 세션 기록이 잘 저장되었는지 확인합니다. store에는 세션 ID를 키로 하여, 해당 세션의 대화 기록이 저장됩니다.

In [8]:
# conversation.predict(input="진희는 강아지를 한마리 키우고 있습니다.")
# conversation.predict(input="영수는 고양이를 두마리 키우고 있습니다.")
# conversation.predict(input="진희와 영수가 키우는 동물은 총 몇마리?")

# 대화 세션 ID 설정
session_id = "unique_session_id_1"

# 대화 진행
output1 = conversation.invoke(
    {"input": "진희는 강아지를 한마리 키우고 있습니다."},
    config={"configurable": {"session_id": session_id}}
)
output2 = conversation.invoke(
    {"input": "영수는 고양이를 두마리 키우고 있습니다."},
    config={"configurable": {"session_id": session_id}}
)
output3 = conversation.invoke(
    {"input": "진희와 영수가 키우는 동물은 총 몇마리?"},
    config={"configurable": {"session_id": session_id}}
)

# 출력 결과 보기
print(f"- output1: [ {output1.content}]\n")
print(f"- output2: [ {output2.content}]\n")
print(f"- output3: [ {output3.content}]\n")
# 메모리 보기
print(f"- store: [{store}]\n")

- output1: [ 진희는 강아지를 키우고 있군요! 강아지와 함께하는 시간은 정말 소중하고 즐거운 경험이죠. 진희의 강아지는 어떤 품종인가요? 그리고 어떤 활동을 함께 하며 시간을 보내나요?]

- output2: [ 영수는 고양이를 두 마리 키우고 있군요! 고양이는 독립적이면서도 애교가 많은 동물이라서 함께 지내는 재미가 쏠쏠하죠. 두 마리의 고양이는 어떤 품종인가요? 그리고 서로 잘 지내나요? 고양이와의 일상 이야기도 궁금합니다!]

- output3: [ 진희는 강아지를 한 마리 키우고 있고, 영수는 고양이를 두 마리 키우고 있습니다. 따라서 진희와 영수가 키우는 동물의 총 수는 1 + 2 = 3마리입니다.]

- store: [{'unique_session_id_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='진희는 강아지를 한마리 키우고 있습니다.'), AIMessage(content='진희는 강아지를 키우고 있군요! 강아지와 함께하는 시간은 정말 소중하고 즐거운 경험이죠. 진희의 강아지는 어떤 품종인가요? 그리고 어떤 활동을 함께 하며 시간을 보내나요?', response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 21, 'total_tokens': 76}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea1d0a67-f5da-4388-937d-1307bc7ec39a-0', usage_metadata={'input_tokens': 21, 'output_tokens': 55, 'total_tokens': 76}), HumanMessage(content='영수는 고양이를 두마리 키우고 있습니다.'), AIMessage(co